In [54]:
import jax
from jax import numpy as jnp
import matplotlib.pyplot as plt
import ces
import balance

In [55]:
def loss(Z, rho, s, h, Z0, Z_target):
    n, m = Z.shape
    n, m = m, n - m
    W = ces.W(Z, rho)
    p = ces.balance_prices(W, rho, s, h)
    J = ces.J(p, W, rho)
    Z_pred = balance.J_to_Z(J, Z0) 
    return jnp.trace(jnp.transpose(Z_pred - Z_target) @ (Z_pred - Z_target))   

In [56]:
n, m = 33, 3
key = jax.random.PRNGKey(0)
tables = balance.read_NIOT('NIOTS/RUS_NIOT_nov16.xlsx')
Z_target = tables[14][:,:n]                           
Z0 = jnp.sum(tables[14][:,n:], axis = 1)[:n]

In [57]:
Z = tables[14][:,:n]
rho = jax.random.uniform(key, (n,), minval= -0.9, maxval = -0.1)
s = jnp.ones(m)
h = 0.0 * jnp.ones(n)

In [58]:
T = jax.jacfwd(loss, argnums = [0,1])(Z, rho, s, h, Z0, Z_target)

In [59]:
T[1]

Array([ 2.9939506e-03, -1.2012524e-02,  4.4911294e-03, -2.4882393e-04,
       -3.6331522e+00,  1.1300843e-05, -3.3541517e-03,  5.7494303e-04,
       -8.4816199e-04, -1.4750770e-03, -1.2880607e-01, -1.7513130e-03,
       -8.9151859e-03, -4.4093855e-02, -3.5544581e-04,  1.1321069e-01,
        8.6962637e-03, -1.9829508e-03, -9.1035374e-02,  2.1323340e-02,
        6.3107954e-03, -8.0163532e-05,  1.7459542e-04, -7.3176611e-04,
       -6.7639330e-06, -3.9403550e-03, -1.0161088e-03, -8.3046965e-02,
       -1.1229251e-01, -9.0770470e-03, -2.8784976e-03, -4.4665602e-04,
       -5.7338632e-04], dtype=float32)